# 14-9. 프로젝트 - Movielens 영화 추천 실습


### 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
data = pd.merge(ratings, movies)

In [7]:
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
using_cols = ['user_id', 'title', 'counts']
data = data[using_cols]
data.head(10)

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
5,18,One Flew Over the Cuckoo's Nest (1975),4
6,19,One Flew Over the Cuckoo's Nest (1975),5
7,24,One Flew Over the Cuckoo's Nest (1975),5
8,28,One Flew Over the Cuckoo's Nest (1975),3
9,33,One Flew Over the Cuckoo's Nest (1975),5


### 2) 분석

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
# 유니크한 영화 개수
data['title'].nunique()

3628

In [11]:
# 유니크한 사용자 수
data['user_id'].nunique()

6039

In [12]:
# 인기 있는 영화 30개(인기순)
title_count = data.groupby('title')['counts'].count()
print(title_count.sort_values(ascending=False).head(30))

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [13]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가


In [14]:
my_favorite = ['The Grand Budapest Hotel (2014)', 'La La Land (2016)', 'Batman (1989)', 'Love Letter (1999)', 'Love Actually (1994)']
my_playlist = pd.DataFrame({'user_id':['dahyun']*5, 'title':my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['dahyun']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,dahyun,The Grand Budapest Hotel (2014),5
1,dahyun,La La Land (2016),5
2,dahyun,Batman (1989),5
3,dahyun,Love Letter (1999),5
4,dahyun,Love Actually (1994),5


In [16]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['dahyun'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['La La Land (2016)'])

6039
3629


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,3628,5
1,6039,3629,5
2,6039,527,5
3,6039,3630,5


### 4) CSR matrix

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3632 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose)
csr_data_transpose = csr_data.T
csr_data_transpose

<3632x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보자


In [25]:
dahyun, batman_1989 = user_to_idx['dahyun'], title_to_idx['Batman (1989)']
dahyun_vector, batman_1989_vector = als_model.user_factors[dahyun], als_model.item_factors[batman_1989]

In [26]:
dahyun_vector

array([-0.3472907 , -0.10169961, -0.10396954,  0.17160471,  0.02720726,
        0.10140419,  0.01416236,  0.37942934, -0.6577199 ,  0.07620087,
       -0.15739474, -0.08593126, -0.00998456,  0.23477954, -0.14944914,
        0.088066  , -0.30947065, -0.14796934,  0.11304435,  0.03783915,
        0.05864503, -0.17066197,  0.06726594, -0.10423675,  0.1337209 ,
        0.0649988 ,  0.05373608, -0.29043293,  0.44232383, -0.00460039,
       -0.05184741,  0.05349186, -0.23910688,  0.15432294,  0.00229462,
        0.15986788,  0.05850807, -0.03903748, -0.09570229, -0.17664884,
        0.25844634, -0.27898532,  0.27209604,  0.12834921,  0.37095872,
        0.02607471,  0.05432711, -0.26442412, -0.11486506, -0.5641604 ,
       -0.02354352, -0.15908934, -0.10630905, -0.1454567 ,  0.00941899,
        0.17760453,  0.0024138 ,  0.0268874 ,  0.07420292,  0.25768661,
        0.05760914,  0.1689584 ,  0.10268547, -0.05730515,  0.16901128,
        0.22827418,  0.01602782,  0.10340343, -0.24667417,  0.11

In [27]:
batman_1989_vector

array([-0.02486442, -0.0006894 , -0.00641766,  0.02224336,  0.01206213,
        0.00927673,  0.00178849,  0.0258678 , -0.04529048,  0.01586276,
       -0.00981082, -0.00449686,  0.003029  ,  0.02910958, -0.00925046,
        0.01078516, -0.01710317, -0.00792499,  0.01667349,  0.00695921,
        0.00269209, -0.01260302,  0.00858963,  0.00436588,  0.01122101,
        0.01098116,  0.00507234, -0.02081628,  0.03618029,  0.00623279,
        0.00252319,  0.01113757, -0.01533407,  0.01546565,  0.0104337 ,
        0.02269539,  0.00550923,  0.00208227, -0.00608794, -0.00693796,
        0.03066031, -0.01493874,  0.02812309,  0.02114453,  0.03358263,
        0.00840397,  0.00199413, -0.01210006, -0.00035873, -0.03166233,
        0.00023533, -0.00289988,  0.00111551,  0.003104  ,  0.0030011 ,
        0.01370822,  0.00743224, -0.00084599,  0.01525041,  0.02461143,
        0.005805  ,  0.01551217,  0.00833473, -0.0019717 ,  0.02114505,
        0.01787319,  0.01193681,  0.00410092, -0.0169731 ,  0.01

In [28]:
# dahyun과 batman_1989를 내적하는 코드
np.dot(dahyun_vector, batman_1989_vector)

0.3419883

In [29]:
btf = title_to_idx['Back to the Future (1985)']
btf_vector = als_model.item_factors[btf]
np.dot(dahyun_vector, btf_vector)

-0.0013495475

### 7) 내가 좋아하는 영화와 비슷한 영화 추천받기

In [30]:
favorite_title = 'Toy Story (1995)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(40, 1.0),
 (50, 0.78318685),
 (4, 0.57993954),
 (33, 0.5739332),
 (322, 0.55829334),
 (110, 0.5444947),
 (330, 0.44358698),
 (10, 0.44127533),
 (20, 0.41895303),
 (255, 0.41239083),
 (160, 0.35536197),
 (126, 0.35079026),
 (32, 0.33249378),
 (16, 0.3251607),
 (545, 0.32376054)]

In [31]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 "There's Something About Mary (1998)",
 'Forrest Gump (1994)',
 'Shakespeare in Love (1998)',
 'Hercules (1997)',
 'Tarzan (1999)',
 'Santa Clause, The (1994)']

In [32]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [33]:
get_similar_title('Aladdin (1992)')

['Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Little Mermaid, The (1989)',
 'Toy Story (1995)',
 'Hunchback of Notre Dame, The (1996)',
 'Hercules (1997)',
 'Mulan (1998)',
 'Anastasia (1997)',
 'Tarzan (1999)']

In [34]:
get_similar_title('Mulan (1998)')

['Mulan (1998)',
 'Tarzan (1999)',
 'Hunchback of Notre Dame, The (1996)',
 'Antz (1998)',
 'Rescuers Down Under, The (1990)',
 'Hercules (1997)',
 'Anastasia (1997)',
 'Lion King, The (1994)',
 'Balto (1995)',
 'Pocahontas (1995)']

### 8) 내가 좋아할만한 영화 추천받기

In [36]:
user = user_to_idx['dahyun']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(172, 0.20621455),
 (188, 0.20210585),
 (57, 0.15749347),
 (587, 0.146943),
 (637, 0.12575515),
 (692, 0.12410906),
 (515, 0.120865665),
 (325, 0.118165895),
 (201, 0.1156298),
 (648, 0.113129616),
 (526, 0.11246279),
 (184, 0.109725684),
 (945, 0.104972556),
 (583, 0.10220559),
 (928, 0.10184795),
 (6, 0.101338774),
 (674, 0.10059305),
 (131, 0.09930436),
 (763, 0.0958211),
 (120, 0.09532987)]

In [37]:
[idx_to_title[i[0]] for i in title_recommended]

['Indiana Jones and the Last Crusade (1989)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Untouchables, The (1987)',
 'Batman Returns (1992)',
 'Abyss, The (1989)',
 'Blues Brothers, The (1980)',
 'Batman Forever (1995)',
 'Romancing the Stone (1984)',
 'Who Framed Roger Rabbit? (1988)',
 'Lethal Weapon (1987)',
 'Superman (1978)',
 'Crocodile Dundee (1986)',
 'Highlander (1986)',
 'Star Trek: The Wrath of Khan (1982)',
 'Robocop (1987)',
 'Ben-Hur (1959)',
 'Back to the Future Part II (1989)',
 'Rocky (1976)',
 'Superman II (1980)',
 'Raiders of the Lost Ark (1981)']

In [38]:
jones = title_to_idx['Indiana Jones and the Last Crusade (1989)']
explain = als_model.explain(user, csr_data, itemid=jones)

In [39]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Batman (1989)', 0.20041810107186894),
 ('Love Letter (1999)', 0.0009930102201689424),
 ('La La Land (2016)', 0.0008973214619954597),
 ('The Grand Budapest Hotel (2014)', 0.0008960245784376331),
 ('Love Actually (1994)', 0.0008796962455809536)]